# Einsendeaufgabe 3: Numba (100 Punkte)

In diesem Abschnitt möchten wir unsere Applikationen mit Numba beschleunigen.

## Mandelbrot-Set (10 Punkte)

Wir beginnen mit einem klassischen Beispiel für Parallelisierung: dem Mandelbrot-Set.

Die Mandelbrot-Menge wird durch die Gleichung $z_{n+1} = z_n^2 + c$ beschrieben. In dieser Gleichung sind $c$ und $z$ komplexe Zahlen, und $n$ ist Null oder eine positive ganze Zahl. Ausgehend von $z_0=0$, gehört $c$ zur Mandelbrot-Menge, wenn der Absolutwert von $z_n$ niemals größer als eine bestimmte Zahl wird (diese Zahl ist abhängig von $c$), egal wie groß $n$ wird.

Das bedeutet, eine komplexe Zahl $c$ ist ein Element der Mandelbrotmenge, wenn die Beträge der mit diesem $c$ berechneten $z_{n}$ nicht unbegrenzt wachsen, unabhängig davon, wie groß $n$ wird. Es lässt sich leicht zeigen, dass der Betrag von $z_{n}$ unbegrenzt wächst, sobald ein $z_{n}$ mit $|z_{n}|>2$ auftritt.

Bei der Berechnung des Mandelbrot-Sets legt man oft eine obere Grenze für die maximalen Iterationen fest, um zu testen, ob diese überschritten wird.

Ein einfacher Algorithmus zur Bestimmung, ob ein Wert zum Mandelbrot-Set gehört, sieht folgendermaßen aus:

In [ ]:
import numba
import numpy as np


def escape_time(p, maxiter):
    """Perform the Mandelbrot iteration until it's clear that p diverges
    or the maximum number of iterations has been reached.

    Parameters
    ----------
    p: complex
        point in the complex plane
    maxiter: int
        maximum number of iterations to perform before p is considered in
        the Mandelbrot set.
    """
    z = 0j
    for i in range(maxiter):
        z = z**2 + p
        if abs(z) > 2:
            return i
    return maxiter


a = np.array([0.2 + 0.2j])
escape_time(a, 400)

Die Zahl $0.2+0.2j$ liegt also in der Mandelbrot Menge, da wir nach 400 Iterationen noch nicht die Abbruchbedingung erfüllen.  Allerdings ist es mühsam, jede Zahl einzeln zu testen. Wir erstellen also ein 2-D Feld mit vielen Komplexen Zahlen $|z|<2$: 

In [ ]:
real_param = (-1.5, 1.1, 1500)
imag_param = (-1.5j, 1.1j, 1500)
XX, YY = np.meshgrid(np.linspace(*real_param), np.linspace(*imag_param))
P = XX + YY

<!-- BEGIN QUESTION -->

**Aufgabe:**  
- Vektorisieren Sie die Funktion mit `np.vectorize`
- Vektorisieren Sie die Funktion mit `numba.vectorize`
- Testen Sie, ob der Parallel-Parameter funktioniert und ob er Performance-Vorteile bringt
- Vergleichen Sie die Laufzeit der drei Versionen und stellen Sie den Vergleich graphisch dar

_Points:_ 7

In [ ]:
mandelbrot_set_np_vectorized = ...
mandelbrot_set_numba_vectorized = ...
mandelbrot_set_numba_vectorized_par = ...
...

In [ ]:
maxiter = 200
...

<!-- END QUESTION -->

Mit *plt.imshow(M)* können Sie sich die Mandelbrotmenge $M$ ansehen: 

In [ ]:
M = mandelbrot_set_numba_vectorized_par(P, maxiter)
plt.imshow(M)

<!-- BEGIN QUESTION -->

**Aufgabe:** Vergleichen Sie nun die drei Versionen mit Scalene. Verwenden sie eine Sampling-Rate von $0.0001$

_Points:_ 3

In [ ]:
%load_ext scalene

In [ ]:
%%scalene --cpu-sampling-rate 0.0001
...

<!-- END QUESTION -->

## Springende Bälle, Teil 2 (60 Punkte)
Wir wollen die Simulation mit mehreren Bällen verbessern und optimieren. 

Jetzt werden zusätzlich Kollisionen zwischen den Bällen beachtet.
Dafür wollen wir zunächst berechnen, ob die Bälle in einem Zeitschritt kollidieren. Eine genaue Berechnung ist hier sehr schwer, daher können wir dies nur näherungsweise machen. Wir müssen dazu auch folgende Annahmen machen:

1. Die Bälle haben einen Radius von $r$, alle Bälle sind gleich groß 
2. Der Zeitschritt $\Delta t$ und die Geschwindigkeit $\$v$ der Bälle sind so gewählt, dass ein Ball maximal viermal seinen Radius in einem Zeitschritt zurücklegt, also $v*\Delta t <= 4*r$


Wir berechnen nun in jeden Zeitschritt zunächst 3 zusätzliche Zwischenzeitschritte:

$ c_0 = c_0$

$c_1 = c_0+0.25\Delta t *v$

$c_3 = c_0+0.5\Delta t *v$

$c_4 = c_0+0.75\Delta t * v$

(Der nächste Schritt wäre $\Delta t$, dieser wird aber in der nächsten Iteration beachtet.)


Wir betrachten nun jeweils für 2 Bälle, ob sie sich in einem der Zwischenzeitschritte näher als zweimal dem Radius $r$ kommen. Wenn sie das tun, gibt es eine Kollision.

<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Funktion, die dies für zwei Bälle testet und zunächst `True` oder `False` zurück gibt. Für die Berechnung des Abstands können sie die Funktion `np.linalg.norm` verwenden.

_Points:_ 10

In [ ]:
def will_collide(c1, v1, c2, v2, r, dt):
    ...

<!-- END QUESTION -->

Um die Geschwindigkeiten der Bälle nach einer Kollision zu berechnen, können wir die Erhaltung des Impulses und die Erhaltung der kinetischen Energie heranziehen, unter der Annahme, dass es sich um eine vollkommen elastische Kollision handelt. Bei einer solchen Kollision bleiben sowohl der Gesamtimpuls als auch die gesamte kinetische Energie des Systems erhalten. Die Berechnungen werden allerdings etwas komplexer, wenn wir Richtungen berücksichtigen.

#### Geschwindigkeiten nach der Kollision

Für zwei Bälle mit Massen $m_1$ und $m_2$, den Anfangsgeschwindigkeiten $v_1$ und $v_2$ und den Endgeschwindigkeiten $v_1'$ und $v_2'$ gelten folgende Gleichungen basierend auf dem Impulserhaltungssatz und der Erhaltung der kinetischen Energie:

1. **Impulserhaltung:**
   $m_1 v_1 + m_2 v_2 = m_1 v_1' + m_2 v_2'$

2. **Erhaltung der kinetischen Energie:**
   $\frac{1}{2} m_1 v_1^2 + \frac{1}{2} m_2 v_2^2 = \frac{1}{2} m_1 {v_1'}^2 + \frac{1}{2} m_2 {v_2'}^2$

Diese Gleichungen müssen vektoriell gelöst werden, da die Geschwindigkeiten Richtungen haben.

Die Lösungen für die Geschwindigkeiten nach der Kollision in zwei Dimensionen unter der Annahme einer vollkommen elastischen Kollision und unter Ignorierung der Rotation der Bälle sind:

$v_1' = v_1 - \frac{2 m_2}{m_1 + m_2} \frac{(v_1 - v_2) \cdot (c_1 - c_2)}{\|c_1 - c_2\|^2} (c_1 - c_2)$

$v_2' = v_2 + \frac{2 m_1}{m_1 + m_2} \frac{(v_1 - v_2) \cdot (c_1 - c_2)}{\|c_1 - c_2\|^2} (c_1 - c_2)$

Hierbei ist $(v_1 - v_2) \cdot (c_1 - c_2)$ das Skalarprodukt der Vektoren $v_1 - v_2$ und $c_1 - c_2$, und $\|c_1 - c_2\|$ ist der Betrag des Vektors $(c_1 - c_2)$, welcher den Abstand zwischen den Zentren der Bälle darstellt.


<!-- BEGIN QUESTION -->

**Aufgabe:** Schreiben Sie eine Funktion, welche die neuen Geschwindigkeiten der Bälle NACH einer Kollision berechnet.  
Sie übergeben der Funktion die folgenden Daten.
- Koordinaten ($c_1, c_2$)
- Geschwindigkeiten ($v_1, v_2$)
- Massen ($m_1, m_2$)

Für die Berechnung des Betrags können Sie wiederum die Funktion `np.linalg.norm` verwenden.

_Points:_ 10

In [ ]:
def collision(c1, c2, v1, v2, m1, m2):
    """Elastischer Stoß zwischen den Bällen 1 und 2."""
    ...
    return v1_new, v2_new


def test_collision(case):
    # Online simulator
    # https://phet.colorado.edu/sims/html/collision-lab/latest/collision-lab_all.html
    if case == "x":
        c1 = np.array([1, 1])
        c2 = np.array([2, 1])
        v1 = np.array([1, 0])
        v2 = np.array([-1, 0])
    if case == "y":
        c1 = np.array([1, 1])
        c2 = np.array([1, 2])
        v1 = np.array([0, 1])
        v2 = np.array([0, -1])
    if case == "diagonal":
        c1 = np.array([1, 1])
        c2 = np.array([2, 2])
        v1 = np.array([1, 1])
        v2 = np.array([0, 0])
    if case == "custom":
        c1 = np.array([1, 1])
        c2 = np.array([2, 3])
        v1 = np.array([1, 1])
        v2 = np.array([0, -1])
    v1_new, v2_new = collision(c1, c2, v1, v2, 1, 1)
    assert np.allclose(np.abs(v2 - v1), np.abs(v2_new - v1_new))
    print(f"{v1=}, {v2=}, {v1_new=}, {v2_new=}")

    # different mass
    v1_new, v2_new = collision(c1, c2, v1, v2, 1, 20)
    assert np.allclose(np.abs(v2 - v1), np.abs(v2_new - v1_new))
    print(f"{v1=}, {v2=}, {v1_new=}, {v2_new=}")


test_collision(case="x")
test_collision(case="y")
test_collision(case="diagonal")
test_collision(case="custom")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Nun wollen wir eine Funktion implementieren, die alle möglichen Kollisionen in einem Zeitschritt erfasst und die neuen Geschwindigkeiten der Bälle berechnet – und zwar möglichst effizient. 

Dafür sollten Sie die folgenden Dinge beachten:
1. Es macht nicht für alle Ballpaare sinnvoll alle vier Zwischenzeitschritte zu berechnen. Testen Sie daher zunächst, ob sich zwei Bälle bereits näher als $ 8 \cdot r $ sind.
2. Sie können annehmen, dass sich der Ball bis zum Zwischenschritt, in dem die Kollision berechnet wird, mit der alten Geschwindigkeit bewegt, danach mit der neuen. Implementieren Sie dies entsprechend. Achten Sie auch darauf, dass ein Ball, für den eine Kollision berechnet wurde, im selben Zeitschritt keine vollständige Bewegung für $ \Delta t $ mehr macht.
3. Es kann leicht passieren, dass zwei Bälle, die in einem Zeitschritt kollidieren, es nicht schaffen, sich während des Zeitschritts weit genug aufeinander zu bewegen, sodass sie im folgenden Zeitschritt wieder miteinander kollidieren. Das kann dann dazu führen, dass die Bälle sich umkreisen. Um das zu verhindern, verwenden wir eine Maske (`mask`), die dafür sorgt, dass zwei Bälle nicht in zwei (oder mehr) aufeinanderfolgenden Zeitschritten miteinander kollidieren können.

Anmerkung: Die Situation, dass mehrere Bälle gleichzeitig, in einem Zeitschritt kollidieren, kann hier vereinfacht ignoriert werden.

_Points:_ 10

In [ ]:
def get_all_collisions(c, v, m, r, dt, mask):
    ...

In [ ]:
import numpy as np

# Anfangsbedingungen (Vorschlag)
g = np.array([0, -9.81], dtype=np.float64)
xmax = 1000
ymax = 1000
steps = 2000
dt = xmax * 1e-5
# Anzahl Bälle
N = 20

# Durchmesser Ball
r = xmax * dt
print(r)
rng = np.random.default_rng(seed=123)


# Startpositionen
def init(n=N):
    c = xmax * rng.random((N, 2))
    # Startgeschwindigkeiten, max.
    v = np.array([xmax, ymax]) * rng.random((N, 2))
    # Masse
    m = np.full(N, 0.05)
    mask = np.full((N, N), 0, np.int32)
    return c, v, m, mask

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Implementieren Sie nun einen vollständigen Zeitschritt mit Kollisionen. 

_Points:_ 10

In [ ]:
def timestep_with_collisions(c, v, m, dt, g, xmax, ymax, mask):
    """
    Simulate a timestep for multiple balls with collisions.

    Parameters:
        c: ...
        v: ...
        m: Masses
        dt: The duration of a single timestep
        g: Gravitation
        xmax: Box limit in x direction
        ymax: Box limit in y direction
    """
    ...

In [ ]:
c, v, m, mask = init()
timestep_with_collisions(c, v, m, dt, g, xmax, ymax, mask)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Animieren Sie die Bewegung der Bälle.

Tipp: Trennen Sie Simulation und Visualisierung und visualisieren Sie z.B. nur jeden 100. Simulationsschritt. 

_Points:_ 5

In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Analysieren Sie nun die Simulation mit Scalene. Was hat sich gegenüber der alten Version ohne die Kollisionen verändert? 

_Points:_ 5

_Type your answer here, replacing this text._

In [ ]:
c, v, m, mask = init(10)

In [ ]:
%%scalene --cpu-sampling-rate 0.01
...

In [ ]:
c, v, m, mask = init(300)

In [ ]:
%%scalene --cpu-sampling-rate 0.01
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Versuchen Sie Ihren Code (d.h. `timestep_with_collisions`) zu optimieren, damit er besser skaliert.

Vergleichen Sie danach die Ausführungszeiten beider Varianten für 20 Zeitschritte mit $[100, 200,300]$ Bällen.

Stellen Sie den Vergleich graphisch dar.

_Points:_ 10

In [ ]:
## Optimized CODE
import numba


...

In [ ]:
c, v, m, mask = init()
timestep_with_collisions_numba(c, v, m, dt, g, xmax, ymax, mask)

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

## Optimierung des Gauß-Verfahrens (30 Punkte)

<!-- BEGIN QUESTION -->

**Aufgabe:** Optimieren Sie die Funktionen `lu_decomposition` und `custom_solve` aus der zweiten Einsendeaufgabe mit Hilfe von Numba.

An welcher Stelle ist die Verwendung von `prange` richtig und sinnvoll? 

Vergleichen Sie die Performance vor und nach der Optimierung für Gleichungssysteme der Größe $256\times 256 $ und $512 \times 512$.

Stellen Sie den Vergleich graphisch dar.

_Points:_ 30

In [ ]:
...

In [ ]:
...

<!-- END QUESTION -->

Führen Sie alle Zellen im Notebook aus und speichern Sie es.  
Danach können Sie mit der folgenden Zelle eine HTML-Datei erstellen.  
Eine mögliche `UserWarning` können Sie ignorieren.  
Bitte geben Sie das Notebook als `.ipynb` und `.html` Datei ab.

In [ ]:
#!jupyter nbconvert --to html EA3.ipynb